# Part 1: Training a pretrained model with TF-Object-Detection-API repo

In this notebook, we will train an [SSD mobilnet v2](https://arxiv.org/pdf/1801.04381.pdf) model (pre-trained on the COCO object detection task) to spot Syd in the Olympics Opening Ceremony and stadiums.

![wanted Syd](https://raw.githubusercontent.com/floydhub/object-detection-template/master/images/wanted-syd.jpg)
*Are you able to catch Syd in this image?*

For more info about Tensorflow Object Detection, please refer to [the official docs](https://github.com/tensorflow/models/tree/master/research/object_detection/g3doc).

**IMPORTANT**

Train on a GPU instance otherwise you will get an *Out-Of-Memory Error*.

### Initial Setup
Let's start by importing and installing the packages, setting the training variables and the path where to save the models.

In [1]:
# Install extra-dependency
! pip3 -q install pycocotools

The current configuration is set to run on a single machine and without overwriting the `pipeline.config` file of the model.

In [2]:
import sys
sys.path.append("/data/slim")

import functools
import json
import os
import tensorflow as tf

from object_detection import trainer
from object_detection.builders import dataset_builder
from object_detection.builders import graph_rewriter_builder
from object_detection.builders import model_builder
from object_detection.utils import config_util
from object_detection.utils import dataset_util

from support import get_train_config, get_eval_config

# For reproducibility
from tensorflow import set_random_seed
from numpy.random import seed
seed(2018)
set_random_seed(2018)

## Train

The configuration is defined in the `PIPELINE_CONFING` file, edit this file or the `train_config` variable to modify the training settings. Training for 3700 steps will take about 30-35 minutes.

In [3]:
##################################################
# CONFIGURATION (Local, Multi-GPUs, Distributed) #
##################################################

MASTER = ''  # Name of the TensorFlow master to use
TASK = 0  # task id
NUM_CLONES = 1  # Number of clones to deploy per worker.

# Force clones to be deployed on CPU.  Note that even if 
# set to False (allowing ops to run on gpu), some ops may 
# still be run on the CPU if they have no GPU kernel.
CLONE_ON_CPU = False

WORKER_REPLICAS = 1  # Number of worker+trainer replicas.
PS_TASKS = 0  # Number of parameter server tasks. If None, does not use a parameter server.
TRAIN_DIR = '/data/trained_models/ssdlite_mobilenet_v2_coco_2018_05_09'  # Directory to save the checkpoints and training summaries.

# Path to a pipeline_pb2.TrainEvalPipelineConfig config
# file. If provided, other configs are ignored
PIPELINE_CONFING_PATH = '/data/models/ssdlite_mobilenet_v2_coco_2018_05_09/pipeline.config'
TRAIN_CONFIG_PATH = ''  # Path to a train_pb2.TrainConfig config file.
INPUT_CONFIG_PATH = ''  # Path to an input_reader_pb2.InputReader config file.
MODEL_CONFIG_PATH = ''  # Path to a model_pb2.DetectionModel config file.

# Get configuration for training
(create_input_dict_fn,
model_fn,
train_config,
master,
task,
worker_job_name,
ps_tasks,
worker_replicas,
is_chief,
graph_rewriter_fn) = get_train_config(TASK,
                                     PS_TASKS,
                                     TRAIN_DIR,
                                     PIPELINE_CONFING_PATH,
                                     TRAIN_CONFIG_PATH,
                                     MODEL_CONFIG_PATH,
                                     INPUT_CONFIG_PATH,
                                     WORKER_REPLICAS,
                                     MASTER)


######################################
# HOW TO EDIT TRAIN CONFIG FROM CODE #
######################################

# You can change the setting in this way,
# e.g. for batch size and num_steps
# train_config.batch_size = 4
# train_config.num_steps = 4000

print("Train configuration: \n", '-'*30, '\n', train_config)

Train configuration: 
 ------------------------------ 
 batch_size: 8
data_augmentation_options {
  random_horizontal_flip {
  }
}
data_augmentation_options {
  ssd_random_crop {
  }
}
optimizer {
  rms_prop_optimizer {
    learning_rate {
      exponential_decay_learning_rate {
        initial_learning_rate: 0.004000000189989805
        decay_steps: 10000
        decay_factor: 0.949999988079071
      }
    }
    momentum_optimizer_value: 0.8999999761581421
    decay: 0.8999999761581421
    epsilon: 1.0
  }
}
fine_tune_checkpoint: "/home/yk/Documents/TensorFlow/object-detection-template/models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt"
num_steps: 3700
fine_tune_checkpoint_type: "detection"



In [ ]:
# Training
trainer.train(
  create_input_dict_fn,
  model_fn,
  train_config,
  master,
  task,
  NUM_CLONES,
  worker_replicas,
  CLONE_ON_CPU,
  ps_tasks,
  worker_job_name,
  is_chief,
  TRAIN_DIR,
  graph_hook_fn=graph_rewriter_fn)

Instructions for updating:
Please switch to tf.train.create_global_step
Instructions for updating:
Use the `axis` argument instead
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from /home/yk/Documents/TensorFlow/object-detection-template/models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt
INFO:tensorflow:Starting Q

INFO:tensorflow:global step 97: loss = 7.5120 (3.796 sec/step)
INFO:tensorflow:global step 98: loss = 8.2626 (3.699 sec/step)
INFO:tensorflow:global step 99: loss = 6.6854 (3.647 sec/step)
INFO:tensorflow:global step 100: loss = 7.9276 (3.517 sec/step)
INFO:tensorflow:global step 101: loss = 7.0472 (3.551 sec/step)
INFO:tensorflow:global step 102: loss = 6.8753 (3.548 sec/step)
INFO:tensorflow:global step 103: loss = 8.0001 (3.509 sec/step)
INFO:tensorflow:global step 104: loss = 7.0447 (3.597 sec/step)
INFO:tensorflow:global step 105: loss = 6.5783 (3.498 sec/step)
INFO:tensorflow:global step 106: loss = 7.0607 (3.419 sec/step)
INFO:tensorflow:global step 107: loss = 7.1038 (3.497 sec/step)
INFO:tensorflow:global step 108: loss = 6.8295 (3.629 sec/step)
INFO:tensorflow:global step 109: loss = 8.5673 (3.668 sec/step)
INFO:tensorflow:global step 110: loss = 7.7180 (3.709 sec/step)
INFO:tensorflow:global step 111: loss = 6.3314 (3.709 sec/step)
INFO:tensorflow:global step 112: loss = 6.6

INFO:tensorflow:global step 220: loss = 4.5903 (4.595 sec/step)
INFO:tensorflow:global step 221: loss = 4.3077 (4.439 sec/step)
INFO:tensorflow:global step 222: loss = 4.3351 (5.298 sec/step)
INFO:tensorflow:global step 223: loss = 4.3449 (3.760 sec/step)
INFO:tensorflow:global step 224: loss = 3.8348 (3.769 sec/step)
INFO:tensorflow:Recording summary at step 224.
INFO:tensorflow:global step 225: loss = 3.2878 (4.939 sec/step)
INFO:tensorflow:global_step/sec: 0.249987
INFO:tensorflow:global step 226: loss = 4.2540 (3.717 sec/step)
INFO:tensorflow:global step 227: loss = 3.9334 (3.581 sec/step)
INFO:tensorflow:global step 228: loss = 3.6454 (3.849 sec/step)
INFO:tensorflow:global step 229: loss = 3.7622 (4.040 sec/step)
INFO:tensorflow:global step 230: loss = 4.6771 (3.852 sec/step)
INFO:tensorflow:global step 231: loss = 4.6475 (5.147 sec/step)
INFO:tensorflow:global step 232: loss = 4.2598 (4.710 sec/step)
INFO:tensorflow:global step 233: loss = 3.7217 (4.636 sec/step)
INFO:tensorflow

INFO:tensorflow:global step 342: loss = 3.2840 (3.627 sec/step)
INFO:tensorflow:global step 343: loss = 2.4883 (3.708 sec/step)
INFO:tensorflow:global step 344: loss = 3.1705 (3.622 sec/step)
INFO:tensorflow:global step 345: loss = 2.7568 (3.511 sec/step)
INFO:tensorflow:global step 346: loss = 3.6574 (3.551 sec/step)
INFO:tensorflow:global step 347: loss = 2.7972 (3.571 sec/step)
INFO:tensorflow:global step 348: loss = 3.2377 (3.508 sec/step)
INFO:tensorflow:global step 349: loss = 2.8787 (3.677 sec/step)
INFO:tensorflow:global step 350: loss = 3.7104 (3.865 sec/step)
INFO:tensorflow:global step 351: loss = 2.9719 (3.741 sec/step)
INFO:tensorflow:global step 352: loss = 3.7037 (3.595 sec/step)
INFO:tensorflow:Recording summary at step 352.
INFO:tensorflow:global step 353: loss = 2.3781 (4.821 sec/step)
INFO:tensorflow:global step 354: loss = 3.8237 (3.597 sec/step)
INFO:tensorflow:global step 355: loss = 3.8286 (3.681 sec/step)
INFO:tensorflow:global step 356: loss = 2.6006 (3.616 sec

INFO:tensorflow:global step 468: loss = 2.7190 (3.708 sec/step)
INFO:tensorflow:global step 469: loss = 2.0577 (3.832 sec/step)
INFO:tensorflow:global step 470: loss = 2.4873 (3.712 sec/step)
INFO:tensorflow:global step 471: loss = 2.9302 (3.596 sec/step)
INFO:tensorflow:global step 472: loss = 1.9087 (3.505 sec/step)
INFO:tensorflow:global step 473: loss = 2.1524 (3.537 sec/step)
INFO:tensorflow:global step 474: loss = 2.1081 (3.529 sec/step)
INFO:tensorflow:global step 475: loss = 2.2429 (3.521 sec/step)
INFO:tensorflow:global step 476: loss = 2.4725 (3.551 sec/step)
INFO:tensorflow:global step 477: loss = 3.2675 (3.462 sec/step)
INFO:tensorflow:global step 478: loss = 2.4542 (3.405 sec/step)
INFO:tensorflow:global step 479: loss = 2.5842 (3.376 sec/step)
INFO:tensorflow:global step 480: loss = 2.2893 (3.376 sec/step)
INFO:tensorflow:global step 481: loss = 2.1111 (3.443 sec/step)
INFO:tensorflow:global step 482: loss = 1.9497 (3.565 sec/step)
INFO:tensorflow:global step 483: loss = 

...

INFO:tensorflow:global step 1458: loss = 1.1256 (3.525 sec/step)
INFO:tensorflow:global step 1459: loss = 1.2672 (3.504 sec/step)
INFO:tensorflow:global step 1460: loss = 1.3599 (3.524 sec/step)
INFO:tensorflow:global step 1461: loss = 1.3862 (3.464 sec/step)
INFO:tensorflow:global step 1462: loss = 1.0662 (3.561 sec/step)
INFO:tensorflow:global step 1463: loss = 0.9490 (3.439 sec/step)
INFO:tensorflow:global step 1464: loss = 1.6903 (3.398 sec/step)
INFO:tensorflow:global step 1465: loss = 1.3748 (3.422 sec/step)
INFO:tensorflow:global step 1466: loss = 1.3788 (3.557 sec/step)
INFO:tensorflow:global step 1467: loss = 2.3225 (3.653 sec/step)
INFO:tensorflow:global step 1468: loss = 2.0036 (3.533 sec/step)
INFO:tensorflow:Saving checkpoint to path trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt
INFO:tensorflow:global step 1469: loss = 1.7747 (4.029 sec/step)
INFO:tensorflow:Recording summary at step 1469.
INFO:tensorflow:global step 1470: loss = 1.3598 (4.637 sec/step)
IN

INFO:tensorflow:global step 1580: loss = 1.9082 (3.415 sec/step)
INFO:tensorflow:global step 1581: loss = 1.6961 (3.424 sec/step)
INFO:tensorflow:global step 1582: loss = 1.2650 (3.577 sec/step)
INFO:tensorflow:global step 1583: loss = 1.4462 (3.714 sec/step)
INFO:tensorflow:global step 1584: loss = 1.4746 (3.649 sec/step)
INFO:tensorflow:global step 1585: loss = 0.9461 (3.680 sec/step)
INFO:tensorflow:global step 1586: loss = 1.2198 (3.598 sec/step)
INFO:tensorflow:global step 1587: loss = 1.8943 (3.469 sec/step)
INFO:tensorflow:global step 1588: loss = 1.3945 (3.410 sec/step)
INFO:tensorflow:global step 1589: loss = 0.8840 (3.473 sec/step)
INFO:tensorflow:global step 1590: loss = 1.3423 (3.577 sec/step)
INFO:tensorflow:global step 1591: loss = 1.4634 (3.618 sec/step)
INFO:tensorflow:global step 1592: loss = 1.3754 (3.612 sec/step)
INFO:tensorflow:global step 1593: loss = 1.2214 (3.659 sec/step)
INFO:tensorflow:global step 1594: loss = 1.5340 (3.542 sec/step)
INFO:tensorflow:global st

INFO:tensorflow:global step 1702: loss = 1.5508 (3.477 sec/step)
INFO:tensorflow:global step 1703: loss = 1.5711 (3.566 sec/step)
INFO:tensorflow:global step 1704: loss = 1.5213 (3.787 sec/step)
INFO:tensorflow:global step 1705: loss = 1.6088 (3.718 sec/step)
INFO:tensorflow:global step 1706: loss = 1.1367 (3.583 sec/step)
INFO:tensorflow:global step 1707: loss = 1.5787 (3.625 sec/step)
INFO:tensorflow:global step 1708: loss = 2.4692 (3.830 sec/step)
INFO:tensorflow:global step 1709: loss = 1.7938 (3.932 sec/step)
INFO:tensorflow:global step 1710: loss = 1.8096 (3.766 sec/step)
INFO:tensorflow:global step 1711: loss = 1.2530 (3.653 sec/step)
INFO:tensorflow:global step 1712: loss = 1.7517 (3.526 sec/step)
INFO:tensorflow:global step 1713: loss = 1.2558 (3.485 sec/step)
INFO:tensorflow:global step 1714: loss = 0.9034 (3.908 sec/step)
INFO:tensorflow:global step 1715: loss = 1.5240 (3.913 sec/step)
INFO:tensorflow:global step 1716: loss = 1.7563 (3.687 sec/step)
INFO:tensorflow:global st

INFO:tensorflow:global step 1824: loss = 1.3111 (5.168 sec/step)
INFO:tensorflow:global step 1825: loss = 1.5611 (3.825 sec/step)
INFO:tensorflow:global step 1826: loss = 1.1997 (4.045 sec/step)
INFO:tensorflow:global step 1827: loss = 1.1105 (4.545 sec/step)
INFO:tensorflow:global step 1828: loss = 1.3238 (4.187 sec/step)
INFO:tensorflow:global step 1829: loss = 1.3474 (3.710 sec/step)
INFO:tensorflow:global step 1830: loss = 1.0854 (3.787 sec/step)
INFO:tensorflow:global step 1831: loss = 2.5270 (4.215 sec/step)
INFO:tensorflow:global step 1832: loss = 1.3838 (4.083 sec/step)
INFO:tensorflow:global step 1833: loss = 1.9854 (3.684 sec/step)
INFO:tensorflow:global step 1834: loss = 1.2839 (3.819 sec/step)
INFO:tensorflow:global step 1835: loss = 1.5706 (3.808 sec/step)
INFO:tensorflow:global step 1836: loss = 0.9106 (4.077 sec/step)
INFO:tensorflow:global step 1837: loss = 0.9404 (4.004 sec/step)
INFO:tensorflow:global step 1838: loss = 1.3482 (3.756 sec/step)
INFO:tensorflow:global st

INFO:tensorflow:global step 1946: loss = 1.6287 (3.855 sec/step)
INFO:tensorflow:global step 1947: loss = 1.2068 (3.709 sec/step)


## Evaluate

Now it's time to evaluate our trained model.

**Note**

Unfortunately a bug prevent the correct output of [IOU(Intersection Over Union) metric](https://en.wikipedia.org/wiki/Jaccard_index). We will fix it as soon as possible.

In [ ]:
from object_detection import evaluator
from object_detection.utils import label_map_util

tf.reset_default_graph()

##############################
# CONFIGURATION (Evaluation) #
##############################

# Directory to write eval summaries to.
EVAL_DIR = '/data/trained_models/ssdlite_mobilenet_v2_coco_2018_05_09/eval' 

# If training data should be evaluated for this job.
EVAL_TRAINING_DATA = False

# Option to only run a single pass of evaluation. 
# Overrides the `max_evals` parameter in the provided config
RUN_ONCE = True

# Directory containing checkpoints to evaluate, typically
# set to `train_dir` used in the training job.
CHECKPOINT_DIR = TRAIN_DIR

EVAL_CONFIG_PATH = ''  # Path to a eval_pb2.TrainConfig config file.
EVAL_INPUT_CONFIG_PATH = ''  # Path to an input_reader_pb2.InputReader config file.
MODEL_CONFIG_PATH = ''  # Path to a model_pb2.DetectionModel config file.

# Get configuration for Evaluation
(create_input_dict_fn,
model_fn,
eval_config,
categories,
graph_rewriter_fn) = get_eval_config(EVAL_DIR,
                                    PIPELINE_CONFING_PATH,
                                    EVAL_CONFIG_PATH,
                                    MODEL_CONFIG_PATH,
                                    EVAL_INPUT_CONFIG_PATH,
                                    EVAL_TRAINING_DATA,
                                    RUN_ONCE)

######################################
# HOW TO EDIT EVAL CONFIG FROM CODE #
######################################

# You can change the setting in this way,
# e.g. for evaluate only once
# eval_config.max_evals = 1

print("Eval configuration: \n", '-'*30, '\n', eval_config)

In [ ]:
# Evaluate
evaluator.evaluate(
    create_input_dict_fn,
    model_fn,
    eval_config,
    categories,
    CHECKPOINT_DIR,
    EVAL_DIR,
    graph_hook_fn=graph_rewriter_fn)

## Exporting the model

In the last section of the notebook, we will export the model to run prediction on images converted into tensors (matrix representation for [height, width, color_channels]).

In [ ]:
# Exporting the model for Evaluation
from google.protobuf import text_format
from object_detection import exporter
from object_detection.protos import pipeline_pb2

slim = tf.contrib.slim
tf.reset_default_graph()

config_override=''
input_shape=None
trained_checkpoint_prefix = os.path.join(TRAIN_DIR, 'model.ckpt-3700') # EDIT: model.ckpt-<iteration>
output_directory='/data/trained_models/ssdlite_mobilenet_v2_coco_2018_05_09_exported'

# Type of input node. Can be one of :
# [`image_tensor`, `encoded_image_string_tensor`,`tf_example`]
input_type='image_tensor' 

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.gfile.GFile(PIPELINE_CONFING_PATH, 'r') as f:
    text_format.Merge(f.read(), pipeline_config)
text_format.Merge(config_override, pipeline_config)
if input_shape:
    input_shape = [
        int(dim) if dim != '-1' else None
        for dim in input_shape.split(',')
    ]
else:
    input_shape = None
exporter.export_inference_graph(input_type, pipeline_config,
                                  trained_checkpoint_prefix,
                                  output_directory, input_shape)

Great! Will we be able to find Syd in never seen images? Let's jump to the  `3_prediction` notebook to find the answer.